In [ ]:
pip uninstall -y qfinuwa

In [ ]:
!pip install qfinuwa

In [1]:
import pandas as pd
from qfinuwa import Indicators, Strategy, Backtester, Plotting

In [37]:
class CustomIndicators(Indicators):
    
    @Indicators.MultiIndicator
    def bollinger_bands(self, stock, BOLLINGER_WIDTH = 2, WINDOW_SIZE = 50):

        mid_price = (stock['high'] + stock['low']) / 2
        rolling_mid = mid_price.rolling(WINDOW_SIZE).mean()
        rolling_std = mid_price.rolling(WINDOW_SIZE).std()

        return {"upper_bollinger": rolling_mid + BOLLINGER_WIDTH*rolling_std,
                "lower_bollinger": rolling_mid - BOLLINGER_WIDTH*rolling_std}
    
    # @Indicators.SingleIndicator
    # def theo_etf(self, stocks):
    #     weights = {
    #         'AAPL': 74235724,
    #         'MSFT': 33656109,
    #         'NVDA': 11813404,
    #         'V': 7278487,
    #         'MA': 4106032,
    #         'AVGO': 1923552,
    #         'CRM': 4730962,
    #         'TXN': 4355505,
    #     }

    #     n_shares = sum(weights.values())
    #     tot = n_shares = 19697965 + 3013530
    #     factor = tot/n_shares

    #     midprices = [(w/n_shares)*stocks[s]['close'] for s,w in weights.items()]

    #     summation = sum(midprices)

    #     return factor*summation

    @Indicators.SingleIndicator
    def pairs_ratio(self, stocks, WINDOW_SIZE = 30):
        vgt_mid_price = (stocks['VGT']['high'] + stocks['VGT']['low']) / 2
        iyw_mid_price = (stocks['IYW']['high'] + stocks['IYW']['low']) / 2
    
        price_ratio = vgt_mid_price / iyw_mid_price

        rolling_mean = price_ratio.rolling(window=WINDOW_SIZE).mean()
        rolling_std = price_ratio.rolling(window=WINDOW_SIZE).std()
        z_score = (price_ratio - rolling_mean) / rolling_std
        return {"z_score" : z_score}

In [36]:
ci = CustomIndicators()
data = ['VGT', 'IYW']
stocks = {d: pd.read_csv('data/' + d + '.csv') for d in data}
z_score = ci.pairs_ratio(stocks)
z_score.describe()

count    431143.000000
mean         -0.026137
std           1.464486
min          -5.561893
25%          -1.646939
50%          -0.049239
75%           1.647012
max           5.294654
dtype: float64

In [12]:
class MyStrategy(Strategy):
    
    def __init__(self, quantity=5):
        self.quantity = quantity
        self.n_buys = 0
        self.n_sells = 0
    
    def on_data(self, prices, indicators, portfolio):

        for stock in portfolio.stocks:
            # Buy if current price is below lower bollinger band
            if(prices['close'][stock][-1] < indicators['lower_bollinger'][stock][-1]):
                portfolio.order(stock, quantity = self.quantity)
                self.n_buys += 1
            
            # Sell if current price is above upper bollinger band
            if(prices['close'][stock][-1] > indicators['upper_bollinger'][stock][-1]):
                portfolio.order(stock, quantity = -self.quantity)
                self.n_sells += 1

    def on_finish(self):
        # Added to results object - access using result.on_finish
        return self.n_buys, self.n_sells

In [38]:
class PairsTrading(Strategy):

    def _init_(self, quantity=5):
        self.quantity = quantity
        self.n_buys = 0
        self.n_sells = 0

    def on_data(self, prices, indicators, portfolio): 
        
        return
        # entry_threshold = 1.5 
        # exit_threshold = 0.5

        # for z_score in z_scores:
        #     if z_score > entry_threshold:
        #         print("Buy VGT and sell IYW")
        #     elif z_score < - entry_threshold:
        #         print("Buy IYW and sell VGT")
        #     if abs(z_score) < exit_threshold:
        #         print("Exit positions")
        #     else:
        #         print("Do nothing")'
    
    def on_finish(self):
        return
        

In [39]:
backtester = Backtester(PairsTrading, CustomIndicators, ['VGT', 'IYW'], data_folder='data', days=30, fee=0.000, delta_limits=200)
backtester

> Precompiling data: 100%|██████████| 431430/431430 [00:03<00:00, 135509.49it/s]


ValueError: Indicator function pairs_ratio must return a dict

In [17]:
# backtester.run(cv=5, start_dates=['24-01-2022', '02-09-2022', '12-11-2021', '06-10-2021', '31-12-2021'])
backtester.run(cv=10)

> Running backtest over 10 samples of 30 days: 100%|██████████| 10/10 [00:04<00:00,  2.28it/s]



{'strategy': {'quantity': 5}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 50}}}

Mean ROI:	13397.596208924539
STD ROI:	44157.20120493155

13/10/2022 -> 11/11/2022:	0.000
19/05/2022 -> 17/06/2022:	7934.133
13/04/2021 -> 12/05/2021:	-48633.209
19/10/2021 -> 17/11/2021:	75878.483
06/05/2021 -> 04/06/2021:	-73960.392
21/09/2021 -> 20/10/2021:	56801.155
16/08/2022 -> 14/09/2022:	42609.246
25/04/2022 -> 24/05/2022:	20177.812
15/11/2022 -> 14/12/2022:	46271.884
29/11/2021 -> 28/12/2021:	6896.851

AVERAGED RESULTS FOR 10 RUNS:
|               |     VGT |     Net |
|---------------|---------|---------|
| n_trades      |  2015.6 |  2015.6 |
| n_buys        |  1010.8 |  1010.8 |
| n_sells       |  1004.8 |  1004.8 |
| gross_pnl     | 13397.6 | 13397.6 |
| fees_paid     |       0 |       0 |
| net_pnl       | 13397.6 | 13397.6 |
| pnl_per_trade | 6.61137 | 6.61137 |

In [10]:
backtester.run_grid_search(
    strategy_params={'quantity': [1, 5, 10]},
    indicator_params={'bollinger_bands': {'BOLLINGER_WIDTH': [1, 2], 'WINDOW_SIZE': [10, 50, 100]}},
    cv=10)

> Backtesting the across the following ranges:
Agorithm Parameters {'quantity': [1, 5, 10]}
Indicator Parameters {'bollinger_bands': {'BOLLINGER_WIDTH': [1, 2], 'WINDOW_SIZE': [10, 50, 100]}}


Running paramter sweep (cv=10): 100%|██████████| 18/18 [01:04<00:00,  3.59s/it]


Best parameter results:

{'strategy': {'quantity': 1}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 50}}}

Mean ROI:	32978.63621817609
STD ROI:	51268.52381632615

14/07/2021 -> 12/08/2021:	42870.008
08/02/2022 -> 09/03/2022:	-77715.676
19/10/2022 -> 17/11/2022:	45193.670
09/06/2021 -> 08/07/2021:	44388.714
08/12/2021 -> 06/01/2022:	80842.582
26/08/2021 -> 24/09/2021:	52077.063
07/01/2022 -> 04/02/2022:	79876.865
21/10/2021 -> 19/11/2021:	87287.688
25/08/2021 -> 23/09/2021:	-41755.677
16/06/2021 -> 15/07/2021:	16721.127

AVERAGED RESULTS FOR 10 RUNS:
|               |     VGT |     Net |
|---------------|---------|---------|
| n_trades      |  4081.3 |  4081.3 |
| n_buys        |  2083.8 |  2083.8 |
| n_sells       |  1997.5 |  1997.5 |
| gross_pnl     | 36274.8 | 36274.8 |
| fees_paid     |  3296.2 |  3296.2 |
| net_pnl       | 32978.6 | 32978.6 |
| pnl_per_trade | 9.07188 | 9.07188 |